In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve

In [2]:
train_features = pd.read_csv(r'C:\Users\Vaishnavi\Downloads\dataset and all\training_set_features.csv')
train_labels = pd.read_csv(r'C:\Users\Vaishnavi\Downloads\dataset and all\training_set_labels.csv')
test_features = pd.read_csv(r'C:\Users\Vaishnavi\Downloads\dataset and all\test_set_features.csv')

In [3]:
train_data = pd.merge(train_features, train_labels, on='respondent_id')

In [4]:
X = train_data.drop(columns=['xyz_vaccine', 'seasonal_vaccine', 'respondent_id'])
y_xyz = train_data['xyz_vaccine']
y_seasonal = train_data['seasonal_vaccine']
test_data = test_features.drop(columns=['respondent_id'])

In [5]:
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numerical_features = X.select_dtypes(include=[np.number]).columns.tolist()


In [6]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [7]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])



In [8]:
X_train, X_val, y_train_xyz, y_val_xyz, y_train_seasonal, y_val_seasonal = train_test_split(X, y_xyz, y_seasonal, test_size=0.2, random_state=42)


In [9]:
model.fit(X_train, y_train_xyz)
y_pred_xyz = model.predict_proba(X_val)[:, 1]
roc_auc_xyz = roc_auc_score(y_val_xyz, y_pred_xyz)

print(f'ROC AUC for xyz_vaccine: {roc_auc_xyz}')


ROC AUC for xyz_vaccine: 0.8570717671381387


In [10]:
model.fit(X_train, y_train_seasonal)
y_pred_seasonal = model.predict_proba(X_val)[:, 1]
roc_auc_seasonal = roc_auc_score(y_val_seasonal, y_pred_seasonal)

print(f'ROC AUC for seasonal_vaccine: {roc_auc_seasonal}')


ROC AUC for seasonal_vaccine: 0.8551275706017112


In [11]:
model.fit(X, y_xyz)
test_pred_xyz = model.predict_proba(test_data)[:, 1]

model.fit(X, y_seasonal)
test_pred_seasonal = model.predict_proba(test_data)[:, 1]


In [12]:
submission = pd.DataFrame({
    'respondent_id': test_features['respondent_id'],
    'xyz_vaccine': test_pred_xyz,
    'seasonal_vaccine': test_pred_seasonal
})

submission.to_csv('submission.csv',index=False)